In [1]:
import tatqa_utils
import pandas as pd

In [2]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [5]:
res = {}
for idx, item in devdf.iterrows():
    for q in item['questions']:
        answer_type = q['answer_type']
        answer_from = q['answer_from']
        if answer_type not in res:
            res[answer_type] = {}
        if answer_from not in res[answer_type]:
            res[answer_type][answer_from] = 0
        res[answer_type][answer_from] =  res[answer_type][answer_from]  + 1
res

{'span': {'text': 349, 'table-text': 181, 'table': 171},
 'multi-span': {'text': 24, 'table-text': 101, 'table': 92},
 'arithmetic': {'table-text': 205, 'table': 497, 'text': 16},
 'count': {'table': 12, 'table-text': 20}}

In [6]:
pd.DataFrame(res)

,span,multi-span,arithmetic,count
text,349,24,16,NaN
table-text,181,101,205,20.0
table,171,92,497,12.0


In [12]:
res = []
for idx, item in devdf.iterrows():
    for q in item['questions']:
        tlength = len(item['table']['table'])
        paragraphs = " ".join([p['text'] for p in item['paragraphs']])
        plength = len(paragraphs)
        qlength = len(q['question'])
        res.append({'tid':item['table']['uid'], 'qid':q['uid'], 'length': tlength+plength+qlength})
ldf = pd.DataFrame(res)
ldf

,tid,qid,length
0,3ffd9053-a45d-491c-957a-1b2fa0af0570,23801627-ff77-4597-8d24-1c99e2452082,919
1,3ffd9053-a45d-491c-957a-1b2fa0af0570,4960801d-277d-4f79-8eca-c4d0200fa9d6,907
2,3ffd9053-a45d-491c-957a-1b2fa0af0570,593c4388-5209-4462-8b83-b429c8612c25,893
3,3ffd9053-a45d-491c-957a-1b2fa0af0570,f4142349-eb72-49eb-9a76-f3ccb1010cbc,920
4,3ffd9053-a45d-491c-957a-1b2fa0af0570,eb787966-fa02-401f-bfaf-ccabf3828b23,911
...,...,...,...
1663,8db2d2a5-fdcd-45c2-be2f-813cba4fbca7,112727c3-fefd-4d11-9edd-5370981e9c0c,3619
1664,8db2d2a5-fdcd-45c2-be2f-813cba4fbca7,08fbb08c-93c7-4ffc-b92e-5beec6fd7120,3635
1665,8db2d2a5-fdcd-45c2-be2f-813cba4fbca7,ec696158-f93d-4608-903d-a6b9c4e7bfd5,3654
1666,8db2d2a5-fdcd-45c2-be2f-813cba4fbca7,27911d6f-dfe5-487b-bc56-7e5cf16a32e0,3617


In [17]:
ldf['length'].min(), ldf['length'].max(), ldf['length'].mean(), ldf['length'].median()

(132, 8849, 1497.0845323741007, 1065.0)

In [24]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 


llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
llm.invoke('Say Hi!')

AIMessage(content='Hi there! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7fb9742d-0d0d-45ee-97af-3e3c03054ee5-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})